In [1]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense,Activation,Convolution2D,Flatten,MaxPooling2D
from keras.utils import np_utils
from keras.layers import Dropout



Using TensorFlow backend.


In [2]:
ds_X = pd.read_csv('/home/armaan/Hand_Gesture/dataset_X.csv')
ds_Y = pd.read_csv('/home/armaan/Hand_Gesture/dataset_Y.csv')
print ds_X.shape
print ds_Y.shape

(4490, 2500)
(4490, 1)


In [3]:
X_data = np.array(ds_X)
Y_data = np.array(ds_Y)
print X_data.shape
print Y_data.shape
data = np.concatenate((X_data,Y_data),axis=1)
print data.shape

(4490, 2500)
(4490, 1)
(4490, 2501)


In [4]:
X_data = data[:,:-1]
Y_data = data[:,-1]
Y_data = np_utils.to_categorical(Y_data)
print Y_data.shape


(4490, 5)


In [5]:
split = int(0.8*data.shape[0])
X_train = X_data[:split,:]
Y_train = Y_data[:split,:]

X_test = X_data[split:,:]
Y_test = Y_data[split:,:]

In [6]:
print X_train.shape
print Y_train.shape

print X_test.shape
print Y_test.shape

(3592, 2500)
(3592, 5)
(898, 2500)
(898, 5)


In [7]:
X_train = X_train/255.0
X_test = X_test/255.0

In [8]:
X_train = X_train.reshape((-1, 50, 50, 1))
X_test = X_test.reshape((-1, 50, 50, 1))
print X_train.shape
print X_test.shape

(3592, 50, 50, 1)
(898, 50, 50, 1)


In [9]:
model = Sequential()


model.add(Convolution2D(32, kernel_size=(3, 3), input_shape=(50, 50, 1)))
model.add(Activation('relu'))

model.add(Convolution2D(16, kernel_size=(3, 3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(8, (3, 3)))
model.add(Activation('relu'))

model.add(Convolution2D(8, (3, 3)))
model.add(Activation('relu'))

model.add(Flatten())

model.add(Dense(256))
model.add(Dropout(0.25))
model.add(Activation('relu'))

model.add(Dense(5))
model.add(Dropout(0.2))

model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 16)        4624      
_________________________________________________________________
activation_2 (Activation)    (None, 46, 46, 16)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 8)         1160      
_________________________________________________________________
activation_3 (Activation)    (None, 21, 21, 8)         0         
__________

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(X_train, Y_train, epochs=10, batch_size=8, validation_data=[X_test, Y_test])

Train on 3592 samples, validate on 898 samples
Epoch 1/10
3592/3592 [==============================] - 19s 5ms/step - loss: 0.2970 - acc: 0.8828 - val_loss: 1.0971 - val_acc: 0.4042
Epoch 2/10
3592/3592 [==============================] - 18s 5ms/step - loss: 0.1052 - acc: 0.9549 - val_loss: 1.1277 - val_acc: 0.4065
Epoch 3/10
3592/3592 [==============================] - 18s 5ms/step - loss: 0.1032 - acc: 0.9479 - val_loss: 0.9960 - val_acc: 0.4065
Epoch 4/10
3592/3592 [==============================] - 20s 5ms/step - loss: 0.1369 - acc: 0.9393 - val_loss: 1.1545 - val_acc: 0.4065
Epoch 5/10
3592/3592 [==============================] - 17s 5ms/step - loss: 0.0991 - acc: 0.9521 - val_loss: 1.1941 - val_acc: 0.4065
Epoch 6/10
3592/3592 [==============================] - 27s 8ms/step - loss: 0.1068 - acc: 0.9499 - val_loss: 1.1918 - val_acc: 0.4065
Epoch 7/10
3592/3592 [==============================] - 19s 5ms/step - loss: 0.1054 - acc: 0.9427 - val_loss: 1.1200 - val_acc: 0.4065
Epoch 8/

In [12]:
model.predict(X_test,batch_size=50, verbose=0, steps=None)

array([[1.8804387e-09, 1.8314425e-12, 8.6568116e-04, 9.9913436e-01,
        6.8099387e-10],
       [2.9813720e-09, 1.0992901e-13, 4.5437112e-08, 1.0000000e+00,
        4.9065452e-12],
       [1.0820863e-09, 6.8472995e-16, 9.7153952e-12, 1.0000000e+00,
        5.6472639e-14],
       ...,
       [2.3609048e-02, 3.0247497e-03, 1.1736832e-02, 9.5499206e-01,
        6.6373809e-03],
       [2.3609048e-02, 3.0247497e-03, 1.1736832e-02, 9.5499206e-01,
        6.6373809e-03],
       [2.3609048e-02, 3.0247497e-03, 1.1736832e-02, 9.5499206e-01,
        6.6373809e-03]], dtype=float32)

In [13]:
import pickle

In [14]:
with open('model_pickle','wb') as f:
    pickle.dump(model,f)